## Data Reading and Transformation
The data (BCI Arboreal) has a lot of fields which are NaN and a lot of irrelevant variables for my analysis right now I only require location (X, Y, UTM-north, UTM-east) 
Time(timestamp)
Name fields

So I run queries to filter out the above fields and remove any empty data fields from the dataframe

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/Dead-Reckoning Arboreal Species in BCI.csv')
# convert the timestamp column to Unix timestamps
df['timestamp'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.timestamp())

# convert the Unix timestamps to integers
df['timestamp'] = df['timestamp'].astype(int)

/var/folders/2z/k0vctg855l3dq626nd2hf5j40000gn/T/ipykernel_15226/2044101010.py:3: DtypeWarning: Columns (7,9,18,32,37,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Dead-Reckoning Arboreal Species in BCI.csv')


After reading the data. I'm only taking the GPS sensor logs. I tried to merge the records of other sensors (acc / mag). The only reasonable field I thought would be timestamp however the records didn't match good enough so only keeping the GPS data for now

In [2]:
##Considering the GPS data for this study
df_gps = df[df['sensor-type'] == 'gps']
# df_gps.dropna(axis = 1, inplace = True)|
df_extract = df_gps[['utm-easting', 'utm-northing', 'timestamp']]

6 steps for generalizing and summarizing data to make a flow map 

3. extract the centroids (average points) of the point groups and use them as generating points for Voronoi tessellation;
4. use the resulting Voronoi cells as places for place-based division of the trajectories into segments;
5. for each ordered pair of places, aggregate the trajectory segments starting in the first place and ending in the second place;
6. measure the quality of the generalization and improve it if necessary.

### 1. extract characteristic points from the trajectories;


In [3]:
import math

def get_mean(lst):
    return sum(lst) / len(lst)

def spatial_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def angle_between_vectors(v1, v2):
    dot_product = v1[0] * v2[0] + v1[1] * v2[1]
    magnitude_product = math.sqrt(v1[0]**2 + v1[1]**2) * math.sqrt(v2[0]**2 + v2[1]**2)
    return math.degrees(math.acos(dot_product / magnitude_product))

def extract_significant_points(points, min_distance = 3, max_distance = 10, min_angle = 30, min_stop_duration = 100):
    n = len(points)
    C = [points[0]]
    i = 0
    j = 1

    while j < n:
        
        if j == n - 1:
            C.append(points[j])
            break

        d_space_i_j = spatial_distance(points[i][:2], points[j][:2])
        if d_space_i_j >= max_distance:
            C.append(points[j])
            i = j
            j = i + 1
        #Fine
        else:
            k = j + 1
            while k < n and spatial_distance(points[j][:2], points[k][:2]) < min_distance:
                k = k + 1
            #Fine
            if k > j + 1:
                d_time = points[k-1][2] - points[j][2]
                if d_time >= min_stop_duration:
                    C.append(points[j])
                    i = j
                    j = k
                    
            #Fine
                else:
                    x_vals, y_vals, t_vals = zip(*points[j:k])
                    x_ave = get_mean(x_vals)
                    y_ave = get_mean(y_vals)

                    distances = [spatial_distance((x, y), (x_ave, y_ave)) for x, y, _ in points[j:k]]
                    m = j + distances.index(min(distances))

                    C.append(points[m])
                    i = j
                    j = m + 1

            else:
                a_turn = angle_between_vectors((points[i][0] - points[j][0], points[i][1] - points[j][1]),
                                               (points[j][0] - points[k][0], points[j][1] - points[k][1]))
                if a_turn >= min_angle:
                    C.append(points[j])
                    i = j
                    j = k
                else:
                    j += 1

    return C
extracted_points = extract_significant_points(df_extract.to_numpy())


/var/folders/2z/k0vctg855l3dq626nd2hf5j40000gn/T/ipykernel_15226/2157936881.py:12: RuntimeWarning: invalid value encountered in double_scalars
  return math.degrees(math.acos(dot_product / magnitude_product))


### 2. group the extracted points by spatial proximity;


KeyboardInterrupt: 

## 2.1 Visualizing Groups

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
colors = cm.rainbow([i/len(groups) for i in range(len(groups))])
for i, group in enumerate(groups):
    x = [point.x for point in group.members]
    y = [point.y for point in group.members]
    plt.scatter(x, y, color=colors[i], label=f"Group {i+1}")
# plt.legend()
plt.show()

## 2.5 Optimization of obtained points|

### Voronoi cells using group centroids


### Voronoi cells using group centroids and points